Chapter 1 - Multinomial BNs - Paramaterize BNs from Data

Train Survey

Age - young, adult, old
Sex - male, female
Education - high (high-school), university degree
Occupation - emp (employee), self-employed (self)
Residence - city size: small, big
Transportation - car, train, other

In [ ]:
# clear the memory
rm(list = ls())
library(bnlearn)
library(Rgraphviz)
library(gRain)

# set the directory
setwd("~/Dropbox/IAD/BN_Workshop")

# create an empty graph
dag <- empty.graph(nodes = c("A", "S", "E", "O", "R", "T"))

# Add in the edges
dag <- set.arc(dag, from = "A", to = "E") # age influences education
dag <- set.arc(dag, from = "S", to = "E") # sex influences education
dag <- set.arc(dag, from = "E", to = "O") # .... 
dag <- set.arc(dag, from = "E", to = "R") # ....
dag <- set.arc(dag, from = "O", to = "T") # ....
dag <- set.arc(dag, from = "R", to = "T") # ....

#dag
#modelstring(dag)
#nodes(dag)
#arcs(dag)
#class(dag)

plot(dag)

graphviz.plot(dag, layout = "dot", main = "My train")


graphviz.plot(dag, layout = "dot", main = "My train")


In [ ]:
# check for d-separation
dsep(dag, x = "S", y = "R")
dsep(dag, x = "O", y = "R")

# check for paths
# ?path.exists
path.exists(dag, from = "S", to = "R")

# dsep given information in the graph
dsep(dag, x = "S", y = "R", z = "E")


In [ ]:
# Lets Try this another way!
dag2 <- empty.graph(nodes = c("A", "S", "E", "O", "R", "T"))
arc.set <- matrix(c("A", "E", 
                    "S", "E", 
                    "E", "O",
                    "E", "R",
                    "O", "T",
                    "R", "T"), byrow = TRUE, 
                    ncol = 2, dimnames = list(NULL, c("from", "to")))
arc.set
arcs(dag2) <- arc.set
# dag2
# allow us to compare two dags 
# all.equal(dag, dag2)

# if we try to add an edge that results in a cycle - ERROR
# set.arc(dag, from = "T", to = "E")

# if we want to remove and arc
# drop.arc(dag2, from = "O", to = "T")
# graphviz.plot(dag2, layout = "dot") # didn't catch - b/c not written over
# dag2 <- drop.arc(dag2, from = "O", to = "T")
# graphviz.plot(dag2, layout = "dot") # didn't catch - b/c not written over

In [ ]:
# Read in the data
survey <- read.table("survey.txt", header = TRUE, colClasses = "factor")

# define the states
A.lv <- c("young", "adult", "old")
S.lv <- c("M", "F")
E.lv <- c("high", "uni")
O.lv <- c("emp", "self")
R.lv <- c("small", "big")
T.lv <- c("car", "train", "other")

# specify the CPT for every node ("Expert Defined")
A.prob <- array(c(0.30, 0.50, 0.20), dim = 3, dimnames = list(A = A.lv))
# A.prob 

S.prob <- array(c(0.60, 0.4), dim = 2, dimnames = list(S=S.lv))
#S.prob

O.prob <- array(c(0.96, 0.04, 0.92, 0.08), dim = c(2,2), 
                 dimnames = list(O=O.lv, E=E.lv))
#O.prob

R.prob <- array(c(0.25, 0.75, 0.20, 0.80), dim = c(2,2),
               dimnames = list(R = R.lv, E = E.lv))

E.prob <- array(c(0.75, 0.25, 0.72, 0.28, 0.88, 0.12, 0.64, 
                  0.36, 0.70, 0.30, 0.90, 0.10), dim = c(2,3,2), dimnames = list(E=E.lv, A = A.lv, S = S.lv))

T.prob <- array(c(0.48, 0.42, 0.10, 0.56, 0.36, 0.08, 0.58,
                 0.24, 0.18, 0.70, 0.21, 0.09), dim = c(3,2,2), 
                dimnames = list(T=T.lv, O = O.lv, R = R.lv))

cpt <- list(A = A.prob, S = S.prob, E = E.prob, O = O.prob, R = R.prob,
           T = T.prob)
bn <- custom.fit(dag, dist = cpt)
bn

In [ ]:
#graphviz.plot

# carry out some basic 'highlighting'
hlight <- list(nodes = nodes(dag), arcs = arcs(dag), 
               col = "black", textCol = "gray")

pp <- graphviz.plot(dag, highlight = hlight)
renderGraph(pp)




In [ ]:
# carry out some basic 'highlighting' -- with 'emphasis'
# working with the edges first
edgeRenderInfo(pp) <- list(col = c("S~E" = "black", "E~R" = "black"),
                          lwd = c("S~E" = 3, "E~R" = 3))

renderGraph(pp)

# working with the nodes next
nodeRenderInfo(pp) <- list(col = c("S" = "black", "E" = "black", "R" = "black"),
                          textCol = c("S" = "black", "E" = "black", "R" = "black"),
                          fill = c("E" = "grey"))
renderGraph(pp)

#?nodeRenderInfo



In [ ]:
# Visualizing some of the paramaters
bn.mle <- bn.fit(dag, data = survey, method = "mle")
bn.mle$T

bn.fit.barchart(bn.mle$T, main = "This is travel!", xlab = "P(T | R,O)",
                ylab = "")

bn.fit.barchart(bn.mle$R, main = "This is residence!", xlab = "P(R | E)",
                ylab = "")



Part 3 - Inference

In [ ]:
library(gRain)
#?querygrain

junction <- compile(as.grain(bn.mle)) #junction tree built
qg_dat <- querygrain(junction)
names(qg_dat)
qg_dat$T

# set some evidence 
jedu <- setEvidence(junction, nodes = "R", states = "big")
qg_ev <- querygrain(jedu)
qg_ev$T

## Lets try a conditional probability query
# Lets consider the relationship between sex and transportation conditional on edu
# P(S, T | E = high)
jedu2 <- setEvidence(junction, nodes = "E", states = "high")
SxT.cpt.ev <- querygrain(jedu2, nodes = c("S", "T"), type = "joint")
SxT.cpt.ev

SxT.cpt.dat <- querygrain(junction, nodes = c("S", "T"), type = "joint")
SxT.cpt.dat

## Some plotting over the graph
graphviz.chart(bn.mle, grid = TRUE, main = "Original BN")

?as.bn.fit

#graphviz.chart(as.bn.fit(jedu2, including.evidence = TRUE), grid = TRUE, main = "Original BN")


#bn.w.ev <- as.bn.fit(jedu, including.evidence = TRUE)
#graphviz.chart(bn.mle, grid = TRUE, main = "Original BN")
